In [1]:
from tensorflow import keras
import tensorflow as tf
IMG_SIZE = 256

def build_feature_extractor():
    feature_extractor = keras.applications.VGG16(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.vgg16.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-15 09:34:16.277025: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-15 09:34:16.277708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [81]:
import numpy as np
import os
MAX_SEQ_LENGTH = 50
NUM_FEATURES = 512
NUM_CLASSES = 4
checkpoint_path = ''
EPOCHS = 10
train_dir = ''
test_dir = ''
save_dir = ''


In [82]:
# Function for processing data
def process_train_data(X, y, directory, new_saved):
    for label in new_saved:
        print(label)
        dir = os.path.join(directory, label)
        for file in os.listdir(dir):
            arr = np.load(os.path.join(directory, label, file))
            X.append(arr)
            y.append(new_saved.index(label))
    return (X, y)

In [83]:
new_saved = ["milk", "coffee", "door", "dog"]
X_train = []
y_train = []
train_dir = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/MS-ASL/four_var/four_var_training_data"
X_train, y_train = process_train_data(X_train, y_train, train_dir, new_saved)

milk
coffee
door
dog


In [84]:
X_val = []
y_val = []
val_dir = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/MS-ASL/four_var/four_var_validation_data"
X_val, y_val = process_train_data(X_val, y_val, val_dir, new_saved)

milk
coffee
door
dog


In [85]:
(X_val), (y_val)

([array([[[[-0.13725491, -0.5137255 , -0.6392157 ],
           [-0.14509805, -0.5372549 , -0.6627451 ],
           [-0.13725491, -0.54509807, -0.67058825],
           ...,
           [-0.01176471, -0.42745098, -0.58431375],
           [-0.01960784, -0.43529412, -0.5921569 ],
           [-0.04313726, -0.45882353, -0.6156863 ]],
  
          [[-0.13725491, -0.5137255 , -0.6392157 ],
           [-0.14509805, -0.5372549 , -0.6627451 ],
           [-0.12156863, -0.5294118 , -0.654902  ],
           ...,
           [ 0.09019608, -0.44313726, -0.58431375],
           [ 0.08235294, -0.4509804 , -0.5921569 ],
           [ 0.07450981, -0.45882353, -0.6       ]],
  
          [[-0.13725491, -0.52156866, -0.67058825],
           [-0.12156863, -0.52156866, -0.67058825],
           [-0.11372549, -0.5372549 , -0.6784314 ],
           ...,
           [ 0.07450981, -0.4509804 , -0.5921569 ],
           [ 0.07450981, -0.4509804 , -0.5921569 ],
           [ 0.07450981, -0.4509804 , -0.5921569 ]],
  
    

In [114]:
def prepare_all_videos(num_samples, df, labels): #frames: 3d [video, pixel, rgb]


    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx in range(num_samples):
        # Gather all its frames and add a batch dimension.
        # frames = load_video(os.path.join(root_dir, path))
        frames = df[idx]   #f, i, j, rgb
        # frames = frames[None, ...]
        print((num_samples), len(frames))
        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        a = 0
        for frame in frames:
            a += 1
            video_length = frame.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for f in range(length):
                print(idx, a, f)
                temp_frame_features[0, f, :] = feature_extractor.predict(tf.expand_dims(frame,axis=0))

            temp_frame_mask[0, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels

In [ ]:
def get_sequence_model():
    # class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

In [112]:

def run_experiment(train, train_labels, test, test_labels):
    checkpoint = keras.callbacks.ModelCheckpoint(
        checkpoint_path, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print("START")
    history = seq_model.fit(
        [train[0], train[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )
    print("END")
    seq_model.load_weights(save_dir)
    _, accuracy = seq_model.evaluate([test[0], test[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

##load data

In [113]:
train_df = X_train #3d array of frames
train_labels = y_train
test_df = X_val
test_labels = y_val

train_data = prepare_all_videos(len(train_df), train_df, train_labels)  #frame features
test_data = prepare_all_videos(len(test_df), test_df, test_labels)

_, sequence_model = run_experiment(train_data, train_labels, test_data, test_labels)

UnboundLocalError: local variable 'video_length' referenced before assignment